<a href="https://colab.research.google.com/github/kundan111/ML_Basic/blob/master/last_fm_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommendation using lightfm using lastFm data

**Load data **

In [5]:
!pip install lightfm

  Using cached https://files.pythonhosted.org/packages/e9/8e/5485ac5a8616abe1c673d1e033e2f232b4319ab95424b42499fabff2257f/lightfm-1.15.tar.gz
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [8]:
cd recommender_system_py/

/content/recommender_system_py


In [0]:
file_path = 'data/100k_lines_lastfm.tsv'
from scipy.sparse import coo_matrix
# Data to create our coo_matrix
data, row, col = [], [], []

# Artists by id, and users
artists, users = {}, {}

# Read the file and fill variables with data to
# create the matrix and have the artists by id
with open(file_path) as data_file:
    for n, line in enumerate(data_file):
        readable_data = line.split('\t')
        user =           readable_data[0]
        artist_id =      readable_data[1]
        artist_name =    readable_data[2]
#         print(readable_data[2])
        plays =         int(readable_data[3])

        if user not in users:
            users[user] = len(users)

        if artist_id not in artists:
            artists[artist_id] = {
                    'name' : artist_name,
                    'id' : len(artists)
                    }

        # Data for the coo_matrix if the artist was played > 200 times
        if plays > 200:
            data.append(plays)
            row.append(users[user])
            col.append(artists[artist_id]['id'])
coo = coo_matrix((data,(row,col)))

# We return the matrix, the artist dictionary and the amount of users
data = {
        'matrix' : coo,
        'artists' : artists,
        'users' : len(users)
 }



In [0]:
print(len(coo.toarray()))

**Recommend**

**Create model**

In [10]:
from lightfm import LightFM
model = LightFM(loss = 'warp')
model.fit(data['matrix'],epochs = 50, num_threads = 2 )

**Get recommendation**

In [0]:
import numpy as np
def get_recomm(model, coo_mtrx, user_ids):
  n_items = coo_mtrx.shape[0]
  for user in user_ids:
    scores = model.predict(user, np.arange(n_items))
    top_scores = np.argsort(-scores)[:3]
    print('Recomendations for user %s:' % user)
    for x in top_scores.tolist():
         for artist, values in data['artists'].iteritems():
                if int(x) == values['id']:
                    print('   - %s' % values['name'])
    print('\n')

  

**Making Predictions**

In [12]:
  get_recomm(model, data['matrix'], [23, 34, 45])

Recomendations for user 23:
   - death cab for cutie
   - radiohead
   - nine inch nails


Recomendations for user 34:
   - rock universal
   - the beatles
   - nine inch nails


Recomendations for user 45:
   - autechre
   - bonobo
   - boards of canada


